In [82]:
import sys
import os
sys.path.insert(0,os.path.abspath('..'))
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph,MatchBarrels
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from dask import compute,multiprocessing,delayed
import pathlib
import shutil
import glob
import pandas as pd
import vtk
from scipy.spatial import distance
import numpy as np
from sklearn.decomposition import PCA

# Create output folders

In [2]:
input_path_spatial_graphs = '/nas1/Data_Mythreya/MotorCortexProject/V9/Registration_Local/Input_Spatial_Graphs/'
surface_resolution = 100
scaling_correction = False
neuron_translated = False

In [192]:
# create output folder structure
HOME = str(pathlib.Path(input_path_spatial_graphs).parent)
if scaling_correction:
    if neuron_translated:
        output_root = HOME + '/Outputs_{}_{}_{}/'.format(surface_resolution,'Scaling_Corrected','Neuron_Translated')
    else:
        output_root = HOME + '/Outputs_{}_{}/'.format(surface_resolution,'Scaling_Corrected')
else:
    output_root = HOME + '/Outputs_{}/'.format(surface_resolution)
#print(output_root)
pathlib.Path((output_root)).mkdir(exist_ok=True)

output_amira_spatial_graphs = output_root + '1_Amira_Spatial_Graphs/'
pathlib.Path((output_amira_spatial_graphs)).mkdir(exist_ok=True)

output_amira_section_spatial_graphs = output_root + '2_Section_Graphs/'
pathlib.Path((output_amira_section_spatial_graphs)).mkdir(exist_ok=True)

output_corrected_spatial_graphs = output_root + '3_Corrected_Spatial_Graphs/'
pathlib.Path((output_corrected_spatial_graphs)).mkdir(exist_ok=True)

output_surfaces = output_root + '4_Surfaces/'
pathlib.Path((output_surfaces)).mkdir(exist_ok=True)

output_axis_field = output_root + '5_Axis_Field/'
pathlib.Path((output_axis_field)).mkdir(exist_ok=True)

output_neuron_axis = output_root + '6_Neuron_Axis/'
pathlib.Path((output_neuron_axis)).mkdir(exist_ok=True)

output_locally_reg_neurons = output_root + '7_Locally_Registered_Neuron/'
pathlib.Path((output_locally_reg_neurons)).mkdir(exist_ok=True)

#output_locally_reg_neurons_using_neuron_pca_axis = output_locally_reg_neurons + 'Using_Neuron_PCA_Axis/'
#pathlib.Path((output_locally_reg_neurons_using_neuron_pca_axis)).mkdir(exist_ok=True)

#output_locally_reg_neurons_using_dend_axis = output_locally_reg_neurons + 'Using_Dendrite_Axis/'
#pathlib.Path((output_locally_reg_neurons_using_dend_axis)).mkdir(exist_ok=True)

#output_locally_reg_neurons_using_local_avg_axis = output_locally_reg_neurons + 'Using_Local_Avg_Axis/'
#pathlib.Path((output_locally_reg_neurons_using_local_avg_axis)).mkdir(exist_ok=True)

output_gallery = output_root + '8_Gallery_Neurons/'
pathlib.Path((output_gallery)).mkdir(exist_ok=True)

output_neuron_depth_stats = output_root + '9_Neuron_Stats/'
pathlib.Path((output_neuron_depth_stats)).mkdir(exist_ok=True)

output_reg_neuron_hocs = output_root + '10_Registered_Hocs/'
pathlib.Path((output_reg_neuron_hocs)).mkdir(exist_ok=True)

output_features = output_root + '11_Features/'
pathlib.Path((output_features)).mkdir(exist_ok=True)

# Convert ASC to amira

In [ ]:
for file in glob.glob(input_path_spatial_graphs+'*'):
    #if os.path.basename(file) == 'RA20150807_2_Cell_B.ASC':
    if not os.path.isdir(file): 
        sg = AmiraSpatialGraph(file,file_format='ASC')
        sg.write_spatial_graph(output_amira_spatial_graphs+os.path.basename(file)[:-3]+'am')

# Correct spatial graph errors if any

In [ ]:
# split into section graphs
for file in glob.glob(output_amira_spatial_graphs+'/*'):
    #if os.path.basename(file) == 'RA20150807_2_Cell_B.am':
    sg = AmiraSpatialGraph(file,create_section_graphs=True,compute_edge_length=True)
    #print(sg.graph_data.edge_list)
    animal_name = os.path.basename(file)
    pathlib.Path(output_amira_section_spatial_graphs+animal_name).mkdir(exist_ok=True)
    sg.write_all_section_spatial_graphs(output_amira_section_spatial_graphs+animal_name+'/')

In [ ]:
section_graphs_path = output_amira_section_spatial_graphs
graph_path_3d = output_corrected_spatial_graphs
AXIS_DIRECTION = [1,1,1]
if scaling_correction:
    SCALING_CORRECTION = [2.306/2.35,2.309/2.35,1]
else:
    SCALING_CORRECTION = [1,1,1]
    
for section_folder in sorted(glob.glob(section_graphs_path+'*')):
    print(section_folder)
    sg = AmiraSpatialGraph(section_folder+'/1.am',read_header_only=True,generic_graph=True)
    sg_unscaled = AmiraSpatialGraph(section_folder+'/1.am',read_header_only=True,generic_graph=True)
    found_soma = False
    sections_after_soma = 0
    sections_after_wm = 0
    
    for z in range(len(glob.glob(section_folder+'/*.am'))):
        #print(section_folder+'/{}.am'.format(z+1))
        sg_sec = AmiraSpatialGraph(section_folder+'/{}.am'.format(z+1),axis_directions=AXIS_DIRECTION)
        #print(len(sg_sec.graph_data.edge_list))
        sg_sec_unscaled = AmiraSpatialGraph(section_folder+'/{}.am'.format(z+1),axis_directions=AXIS_DIRECTION,)
        #sg_sec.set_scaling(scaling = SCALING_CORRECTION)
        sg_sec.set_z_coord(z*-100,to_neuron=True)
        sg_sec_unscaled.set_z_coord(z*-100,to_neuron=True)
        
        sg_sec.set_scaling(scaling=SCALING_CORRECTION,to_neuron=True)
        sg_sec.graph_data = sg_sec.combine_subgraphs([sg_sec.pia,sg_sec.wm,\
                                                       sg_sec.bvs,sg_sec.barrels.all_rows_graphdata,\
                                                       sg_sec.neuron.all_neurites_subgraphdata])
        
        sg.graph_data = sg.combine_subgraphs([sg.graph_data,sg_sec.graph_data])
        sg_unscaled.graph_data = sg_unscaled.combine_subgraphs([sg_unscaled.graph_data,sg_sec_unscaled.graph_data])
        
#         if len(sg_sec.neuron.soma.vertices)>0:
#             found_soma = True
#         if len(sg_sec.wm.vertices)>0:
#             sections_after_wm = sections_after_wm + 1
#         if found_soma:
#             sections_after_soma = sections_after_soma + 1
        
#         if sections_after_soma > 2 and sections_after_wm > 2:
#             # lets have atleast 2 sections after soma and wm
#             break
    #print(sg.graph_data.edge_pt_coords)
    txmat = [1,0,0,0, 0,-1,0,0, 0,0,-1,0, 0,0,0,1]#get_axis_flip_tx_mat(sg,need_to_flip=True)
    sg.apply_transformation(txmat)
    sg.write_spatial_graph(graph_path_3d+os.path.basename(section_folder))
    
    if scaling_correction and neuron_translated:
        sg_unscaled.apply_transformation(txmat)
        sg_unscaled.write_spatial_graph(graph_path_3d+'_unscaled.am')
        sg = AmiraSpatialGraph(graph_path_3d+os.path.basename(section_folder),)
        sg_unscaled = AmiraSpatialGraph(graph_path_3d+'_unscaled.am')
        os.remove(graph_path_3d+'_unscaled.am')

        # make sure that scaling does not affect soma position 
        # correct soma position is given by the scaled version
        # so tranlate the unscaled neuron to scaled soma position
        target_soma_position = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
        original_soma_position = np.array(sg_unscaled.neuron.soma.edge_pt_coords).mean(axis=0)
        translation = original_soma_position - target_soma_position
        #print(translation)
        txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, -translation[0],-translation[1],-translation[2],1]
        sg_unscaled.apply_transformation_to_labelled_subgraph(sg_unscaled.neuron.all_neurites_subgraphdata,np.reshape(txmat,[4,4]))
        sg_unscaled.graph_data = sg_unscaled.combine_subgraphs([sg_unscaled.pia,sg_unscaled.wm,sg_unscaled.barrels.all_rows_graphdata,sg_unscaled.neuron.all_neurites_subgraphdata])
        sg.graph_data = sg.combine_subgraphs([sg.pia,sg.wm,sg.barrels.all_rows_graphdata,sg_unscaled.neuron.all_neurites_subgraphdata])
        sg.write_spatial_graph(graph_path_3d+os.path.basename(section_folder))

    
#     pia = Surface(pts=sg.pia.edge_pt_coords)
#     pia_hull = pia.create_delunay_surface_3d(return_hull=True)
#     #Surface(polydata=pia_hull).write_surface_mesh(graph_path_3d+'_scaled_pia.vtk')
#     pia_unscaled = Surface(pts=sg_unscaled.pia.edge_pt_coords)
#     pia_unscaled_hull = pia_unscaled.create_delunay_surface_3d(return_hull=True)
#     #Surface(polydata=pia_unscaled_hull).write_surface_mesh(graph_path_3d+'_unscaled_pia.vtk')
    
#     # get soma depth from pia  from unscaled version
#     soma_depth = 0
#     intersec_pt = [0,0,0]
#     soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
#     #print(soma_center)
#     if len(sg_unscaled.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
#         apical_center = np.array(sg_unscaled.neuron.dendrite.apical_dendrite.edge_pt_coords).mean(axis=0)
#         intersec_pt_unscaled,soma_depth_unscaled = Surface(polydata=pia_unscaled_hull).get_vector_intersection_pt(soma_center,apical_center)
#         intersec_pt_scaled,soma_depth_scaled = Surface(polydata=pia_hull).get_vector_intersection_pt(soma_center,apical_center)
#         depth_change = (soma_depth_unscaled - soma_depth_scaled)
#         new_soma_center = pia.create_pt_along_vector_at_given_distance(-depth_change,soma_center,apical_center)
#     else:
#         # if there is no apical then use basal for direction or just leave as it is
#         #basal_center = np.array(sg_unscaled.neuron.dendrite.basal_dendrite.edge_pt_coords).mean(axis=0)
#         #intersec_pt_unscaled,soma_depth_unscaled = Surface(polydata=pia_unscaled_hull).get_vector_intersection_pt(soma_center,basal_center)
#         #intersec_pt_scaled,soma_depth_scaled = Surface(polydata=pia_hull).get_vector_intersection_pt(soma_center,basal_center)
#         #depth_change = (soma_depth_unscaled - soma_depth_scaled)
#         #new_soma_center = pia.create_pt_along_vector_at_given_distance(-depth_change,soma_center,basal_center)
#         new_soma_center = soma_center
#     #Landmarks(pts=[new_soma_center]).write_landmarks(graph_path_3d+'bla.landamarksAscii')
    
#     translation = soma_center - new_soma_center
#     txmat = [1,0,0,0, 0,1,0,0, 0,0,1,0, -translation[0],-translation[1],-translation[2],1]
    
#     sg.apply_transformation_to_labelled_subgraph(sg.neuron.all_neurites_subgraphdata,np.reshape(txmat,[4,4]))
#     sg.graph_data = sg.combine_subgraphs([sg.pia,sg.wm,sg.barrels.all_rows_graphdata,sg.neuron.all_neurites_subgraphdata])
#     sg.write_spatial_graph(graph_path_3d+os.path.basename(section_folder))
    
    

# Create Pia and WM Surfaces

In [ ]:
#TODO: fix divide surface... fails sometimes
# Create Pia and wm surfaces using both 100 and 500 voxel resolution. wm of 500 provides smooth surface to generate good axis field
# pia using voxel size of 100 gives correct soma depth from pia

In [ ]:
@delayed
def create_surface(file,surface_resolution='100'):
    sg = AmiraSpatialGraph(filename=file)
    # flip if required
    #txmat = get_axis_flip_tx_mat(sg)
    #print(txmat)
    #sg.apply_transformation(txmat)
    #sg.write_spatial_graph(root_dst+os.path.basename(file))
    #sg = AmiraSpatialGraph(root_dst+os.path.basename(file))
    #sg_clipped = get_fist_x_z_planes_sg(sg,17)
    #sg_clipped.write_spatial_graph(root_dst+os.path.basename(file)+'_clipped.am')
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(100)))
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
    wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
    
    if pia.surface.GetNumberOfCells()>0 and wm.surface.GetNumberOfCells()>0:
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_100.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_100.vtk')
        
        os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(500)))
        pia_500 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        pia_500.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_500.vtk')
        
        wm_500 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        wm_500.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_500.vtk')
            
#         wm_500_divided = Surface(polydata=wm_500.divide_surface(4,wm_500.surface))
#         if wm_500_divided.surface.GetNumberOfCells() > 0:
#             wm_500_divided.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_500.vtk')
#             flipped_axes = wm_500_divided.create_axis_field(pia,op_sg_name=None,flip_normals=True,return_axes=True,max_allowed_axis_length=4000)
#             axes = wm_500_divided.create_axis_field(pia,op_sg_name=None,flip_normals=False,return_axes=True,max_allowed_axis_length=4000)
#             sg = AmiraSpatialGraph(generic_graph=True)
#             for edge in flipped_axes:
#                 sg.graph_data.add_edge(edge[0],edge[1])
#             for edge in axes:
#                 sg.graph_data.add_edge(edge[0],edge[1])
#             sg.write_spatial_graph(output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am')
            
#         else:
#             # surface division failed
#             flipped_axes = wm.create_axis_field(pia,op_sg_name=None,flip_normals=True,return_axes=True,max_allowed_axis_length=4000)
#             axes = wm.create_axis_field(pia,op_sg_name=None,flip_normals=False,return_axes=True,max_allowed_axis_length=4000)
#             sg = AmiraSpatialGraph(generic_graph=True)
#             for edge in flipped_axes:
#                 sg.graph_data.add_edge(edge[0],edge[1])
#             for edge in axes:
#                 sg.graph_data.add_edge(edge[0],edge[1])
#             sg.write_spatial_graph(output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am')
    else:
        # create a delunay surface as the traditional way has failed
        pia = Surface(pts=sg.pia.edge_pt_coords)
        wm = Surface(pts=sg.wm.edge_pt_coords)
        pia_hull = pia.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm_hull = wm.create_delunay_surface_3d(return_hull=True,output_filename=output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
        
        # clip at the bottom
        shutil.copyfile(src=file,dst=output_surfaces+os.path.basename(file)[:-3]+'_barrels.am')
        pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open.vtk')
        wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open.vtk')
        #wm.create_axis_field(pia,op_sg_name=output_surfaces+os.path.basename(file)[:-3]+'_axis_field.am',flip_normals=False,\
        #                              return_axes=True)

In [ ]:
fn_list_surfs = []
file_names = glob.glob(output_corrected_spatial_graphs+'*.am')
for i in range(len(file_names)):
    #if os.path.basename(file_names[i])=='RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected.am':
    fn_list_surfs.append(create_surface(file_names[i]))
    
compute(fn_list_surfs)

In [ ]:
# check for missing stuff
for file in glob.glob(output_corrected_spatial_graphs+'*.am'):
    if not os.path.exists(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_100.vtk') :
        print(os.path.basename(file)[:-3])
        os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,output_surfaces+os.path.basename(file)[:-3],str(100)))
        pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
        wm = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')

        if pia.surface.GetNumberOfCells()>0 and wm.surface.GetNumberOfCells()>0:
            pia.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_100.vtk')
            wm.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_100.vtk')

            os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
                  format(file,output_surfaces+os.path.basename(file)[:-3],str(500)))
            pia_500 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia.vtk')
            pia_500.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_500.vtk')

            wm_500 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM.vtk')
            wm_500.clip_surface_at_given_z(-100,output_filename = output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_500.vtk')

#  Generate Axis Field

In [ ]:
# Create axis with the smooth wm (voxel size 500) 
file_names = glob.glob(output_corrected_spatial_graphs+'*.am')
for i in range(len(file_names)):
    file = file_names[i]
    #if os.path.basename(file)[:-3] == 'RA20130320_corrected':
    pia_100 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_100.vtk')
    wm_500 = Surface(output_surfaces+os.path.basename(file)[:-3]+'_WM_bottom_open_500.vtk')
    flipped_axes = wm_500.create_axis_field(pia_100,op_sg_name=None,flip_normals=True,return_axes=True,\
                                            max_allowed_axis_length=5000,check_source_double_touch=True)
    axes = wm_500.create_axis_field(pia_100,op_sg_name=None,flip_normals=False,return_axes=True,\
                                    max_allowed_axis_length=5000,check_source_double_touch=True)
    sg = AmiraSpatialGraph(generic_graph=True)
    for edge in flipped_axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    for edge in axes:
        sg.graph_data.add_edge(edge[0],edge[1])
    sg.write_spatial_graph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am')


# Select Neuron Axis

In [85]:
def write_edge(edge,filename):
    axis_sg = AmiraSpatialGraph(generic_graph=True)
    axis_sg.graph_data.add_edge(edge[0],edge[1])
    axis_sg.write_spatial_graph(filename)

    return axis_sg
    

In [176]:
def resample_edge(edge,num_pts_to_be_inserted):
    resampled_edge = []
    #resample_edge.append(edge[0])
    n = np.array(edge[len(edge)-1]) - np.array(edge[0])
    n_unit = n / np.linalg.norm(n)
    edge_len = Vectors().get_vec_length(edge)
    for i in range(num_pts_to_be_inserted):
        resampled_edge.append(i*n_unit*edge_len/num_pts_to_be_inserted+edge[0])
    return edge

In [177]:
def get_pca_direction(apical_dendrite_pts,res=1):
#     resampled_pts = []
#     for edge in apical_dendrite_edges:
#         for i in range(len(edge)-1):
#             length = distance.euclidean(edge[i],edge[i+1])
#             if length > res:
#                 num_pts_to_be_inserted = int(length/res)
#                 resampled_edge = resample_edge(edge,num_pts_to_be_inserted)
#                 for i in range(len(resampled_edge)):
#                     resampled_pts.append(resampled_edge[i])
    
    pca = PCA(n_components=3,)
    transformed_pca = pca.fit_transform(X=apical_dendrite_pts)
    #print(transformed_pca)
    return pca.components_

In [184]:
for file in glob.glob(output_corrected_spatial_graphs+'*.am'):
    #if os.path.basename(file)[:-3] == 'RA20160330_Cell_B':
    print(os.path.basename(file)[:-3])
    pia = Surface(output_surfaces+os.path.basename(file)[:-3]+'_pia_bottom_open_100.vtk')
    axis_field = AmiraSpatialGraph(output_axis_field+os.path.basename(file)[:-3]+'_axis_field.am',generic_graph=True)
    sg = AmiraSpatialGraph(file)
    soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)

    dendrite_pts = []    
    for pt in sg.neuron.dendrite.apical_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)
    for pt in sg.neuron.dendrite.basal_dendrite.edge_pt_coords:
        dendrite_pts.append(pt)   
    dendrite_mean = np.array(dendrite_pts).mean(axis=0)

    # write dendrite for easy visualization
    sg.write_sub_spatial_graph(sg.neuron.dendrite.apical_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_apical_dendrite.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.basal_dendrite,output_neuron_axis+os.path.basename(file)[:-3]+\
                               '_basal_dendrite.am')

    # Avg Local Axis 
    nearby_axes = Vectors().get_nearest_axes_to_pts_within_radius(axis_field.graph_data.edge_list,[soma_center],800)
    nearby_axes_sg = AmiraSpatialGraph(generic_graph=True)
    unit_vecs = []
    for axis in nearby_axes:
        nearby_axes_sg.graph_data.add_edge(axis[0],axis[1])
        unit_vecs.append(Vectors().get_unit_vec(axis[0],axis[1]))
    nearby_axes_sg.write_spatial_graph(output_neuron_axis+os.path.basename(file)[:-3]+'_axis_field_selected.am')
    avg_uv = np.array(unit_vecs).mean(axis=0)
    end_pt = (avg_uv*6000 + soma_center)
    pia_proj_pt_neuron_using_local_avg_axis,soma_depth_using_local_avg = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)

    write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis_local_avg.am')
    soma_depth_using_local_avg = Vectors().get_vec_length([soma_center,pia_proj_pt_neuron_using_local_avg_axis])
    # decide which method of depth measurement to use
    # using the apical dendrite axis works best for L5 and below neurons
    # where as local axis is good for L2/3
    print(soma_depth_using_local_avg)
    if soma_depth_using_local_avg < 500 or len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) == 0:
        write_edge([soma_center,pia_proj_pt_neuron_using_local_avg_axis],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis.am')

    else:
        # Get apical dend axis using pca
        pca_components = get_pca_direction(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
        #Landmarks(pts=resampled_pts).write_landmarks(output_root+'pts')
        end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
        Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
        print(end_pt)
        angle = Vectors().get_angle_between_vectors([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
                         [soma_center,end_pt+soma_center],ignore_opposite_direction=False)
        print(angle)
        if angle < 90:

            # pca direction is correct.. project to pia
            end_pt = Vectors().create_pt_along_vector_at_given_distance(6000,[0,0,0],pca_components[0,:])
            print(end_pt)
            Landmarks(pts=[end_pt+soma_center]).write_landmarks(output_root+'end_pt')
            pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_pca_axis.am')
        else:
            end_pt = Vectors().create_pt_along_vector_at_given_distance(-6000,[0,0,0],pca_components[0,:])
            pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
            write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_pca_axis.am')
        write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis.am')
#     else:
#         # use axis perpendicular to basal pcas (3rd component)
#         pca_components,resampled_pts = get_pca_direction(sg.neuron.dendrite.basal_dendrite.edge_list)
#         end_pt = Vectors().create_pt_along_vector_at_given_distance(5000,[0,0,0],pca_components[2,:])
#         angle = Vectors().get_angle_between_vectors([soma_center,pia_proj_pt_neuron_using_local_avg_axis],\
#                          [soma_center,end_pt],ignore_opposite_direction=False)
#         if angle < 90:
#             # pca direction is correct.. project to pia
#             end_pt = Vectors().create_pt_along_vector_at_given_distance(5000,[0,0,0],pca_components[2,:])
#             pia_proj_pt_neuron,soma_depth  = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
#             write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_pca_axis.am')
#         else:
#             end_pt = Vectors().create_pt_along_vector_at_given_distance(-5000,[0,0,0],pca_components[2,:])
#             pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt+soma_center,soma_center,extrapolation_len=0)
#             write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_pca_axis.am')
#     # Dendrite local axis
#     dend_uv = Vectors().get_unit_vec(soma_center,dendrite_mean)
#     # check which direction to take by comparing with the local avg axis. dendrite center can fall below the soma center
#     end_pt = (dend_uv*6000 + soma_center)
#     pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)
#     if len(pia_proj_pt_neuron) == 0:
#         end_pt = (dend_uv*-6000 + soma_center)
#         pia_proj_pt_neuron,soma_depth = pia.get_vector_intersection_pt(end_pt,soma_center,extrapolation_len=0)

#     write_edge([soma_center,pia_proj_pt_neuron],output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis_along_dendrite.am')



        

RA20130319_2
643.5198524077047
[-2911.5234248   1081.20728347  5133.61686893]
170.78857588188708
RA20130320_corrected
474.72779264413975
RA20130402
1059.8512974130567
[ 3883.78454943  -842.64608196 -4495.12682292]
19.804816584070704
[ 3883.78454943  -842.64608196 -4495.12682292]
RA20130404_1
618.6878175564437
[ 2915.75038997 -1057.40163732 -5136.17576031]
19.59852127581882
[ 2915.75038997 -1057.40163732 -5136.17576031]
RA20130404_2_corrected
824.7992461766248
[ 3233.18945721  -417.88143891 -5037.04884201]
8.214017640834841
[ 3233.18945721  -417.88143891 -5037.04884201]
RA20130507_1_corrected
791.2580362293918
[-2514.26266293  1226.03161202  5308.0438721 ]
172.7929835493611
RA20130507_2_corrected
546.9202276907706
[-1553.21535978  1157.29549691  5678.74890966]
164.81621179693744
RA20130508_1_corrected
434.55881011815933
RA20130513
524.8277309418087
[-4352.41439757  -154.14935364  4127.07243559]
153.60615362368915
RA20130603
1062.7144543375414
[-3593.3741384   -366.86408162  4790.9365521

306.2261329966101
RA20180109_2_Cluster_B_Cell_D_done.am_Segmented_corrected
385.7897848380498
RA20180109_2_Cluster_B_Cell_E_done.am_Segmented_corrected
Label not found !!!
442.0792647071692
RA20180412_2_Cell_A.am_Segmented_corrected
569.0620756323036
[ 2796.78245036 -4749.58031929 -2370.54734522]
11.935012817955485
[ 2796.78245036 -4749.58031929 -2370.54734522]
RA20180412_2_Cell_B.am_Segmented_corrected
460.74810720759785
RA20180412_2_Cell_C.am_Segmented_corrected
325.9749238239031
RA20180412_2_Cell_D.am_Segmented_segmented
371.430751686084
RA20180412_2_Cell_E.am_Segmented_corrected
224.59990297875225
RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected
1724.9096268528747
[ 5877.04248539     9.41714082 -1208.42167408]
19.99774917267755
[ 5877.04248539     9.41714082 -1208.42167408]
RA20150811_4x_.am_corrected_Segmented
851.4556436475702
[-4817.96769192  1983.44442856  2975.42190596]
174.36714176836264
RA20150813_1_Final_Contour_Cell_Merged.am_corrected_Segmented
844.6146607254591
[ 

# Locally Register Neurons

In [115]:
# rotation from a to b
def align_a_to_b(a,b):
    u_a = Vectors().get_unit_vec(a[0],a[1])
    u_b = Vectors().get_unit_vec(b[0],b[1])
    v = np.cross(u_a,u_b)
    s = np.linalg.norm(v)
    c = np.dot(u_a,u_b)
    I = np.array([1,0,0,0,1,0,0,0,1]).reshape([3,3])
    #print(I)
    v_x = np.array([0,-v[2],v[1],v[2],0,-v[0],-v[1],v[0],0]).reshape([3,3])
    R = I + v_x + np.dot(v_x,v_x) * (1/(1+c))
    r_list = []
    for i in range(3):
        for j in range(3):
            r_list.append(R[i,j])
    #print(R)
    return R


In [116]:
def convert_to_transformation_matrix(rot_mat,translation):
    
    tr_mat = []
    k = 0

    for i in range(3):
        for j in range(3):
            tr_mat.append(rot_mat[i,j])
            k = k + 1
        tr_mat.append(0)
        k = k+1
    tr_mat.append(-translation[0])
    tr_mat.append(-translation[1])
    tr_mat.append(-translation[2])
    tr_mat.append(1)
    return tr_mat

In [185]:
# Locally rotate the neuron and spatial graphs using the neuron axis
#for neuron_axis_type in ['Neuron_PCA_Axis','Dend_Axis','Local_Axis']:
for file in glob.glob(output_corrected_spatial_graphs+'*.am'):
    sg = AmiraSpatialGraph(file)
#     if neuron_axis_type == 'Neuron_PCA_Axis':
#         neuron_axis_sg = AmiraSpatialGraph(output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_pca_axis.am',generic_graph=True)
#         output_path = output_locally_reg_neurons_using_neuron_pca_axis
#     elif neuron_axis_type == 'Dend_Axis':
#         neuron_axis_sg = AmiraSpatialGraph(output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis_along_dendrite.am',generic_graph=True)
#         output_path = output_locally_reg_neurons_using_dend_axis
#     else:
    neuron_axis_sg = AmiraSpatialGraph(output_neuron_axis+os.path.basename(file)[:-3]+'_neuron_axis.am',generic_graph=True)
    output_path = output_locally_reg_neurons

    neuron_axis = neuron_axis_sg.graph_data.edge_list[0]

    rot_mat = align_a_to_b(neuron_axis,[[0,0,0],[0,0,1]])
    translation = neuron_axis[1]
    neuron_axis_sg.apply_rotation(rot_mat,translation=translation)
    neuron_axis_sg.write_spatial_graph(output_path+os.path.basename(file)[:-3]+'_Rotated_Axis.am')

    sg.apply_rotation(rot_mat,translation=translation)
    sg.write_spatial_graph(output_path+os.path.basename(file))
    sg = AmiraSpatialGraph(output_path+os.path.basename(file))
    sg.write_sub_spatial_graph(sg.neuron.all_neurites_subgraphdata,output_path+os.path.basename(file)[:-3]+'_Neuron.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.apical_dendrite,output_path+os.path.basename(file)[:-3]+'_Apical.am')
    sg.write_sub_spatial_graph(sg.neuron.dendrite.basal_dendrite,output_path+os.path.basename(file)[:-3]+'_Basal.am')
    sg_dend = AmiraSpatialGraph()
    sg_dend.graph_data = sg_dend.combine_subgraphs([sg.neuron.dendrite.apical_dendrite,sg.neuron.dendrite.basal_dendrite])
    sg_dend.write_spatial_graph(output_path+os.path.basename(file)[:-3]+'_Dendrite.am')

        

Label not found !!!
Label not found !!!
Label not found !!!
Label not found !!!
Label not found !!!


# Make Cell Type Gallery using manual classification

In [186]:
# read the manual cell type csv
manual_types_csv = pd.read_excel('/nas1/Data_Mythreya/MotorCortexProject/cell_types.xlsx',)
manual_types_csv.replace(np.NaN,'',inplace=True)
# write cell type specific neurons in a gallery by shifting the X coord 
for cell_type in ['L2','L3','L5st','L5tt','L6cc','L6ct']:
    cell_type_sg = AmiraSpatialGraph()
    i = 0
    for file in manual_types_csv[manual_types_csv['Manual_Assessment']==cell_type][['date','suffix']].values:
        i = i+1
        #print(file[1])
#         if file[1] is not None:
#             #print(str(file[1]).split('_'))
#             suffix=''
#             for i in range(len(str(file[1]).split('_'))):
#                 suffix = suffix + '*' + str(file[1]).split('_')[i]
            
#         else:
#             suffix = ''
        exp_name = ('RA'+str(file[0])+'*'+str(file[1]))
        print(exp_name)
        #sg_local_avg = AmiraSpatialGraph(glob.glob(output_locally_reg_neurons_using_local_avg_axis+exp_name+'*Dendrite.am')[0])
        #sg_dend_axis = AmiraSpatialGraph(glob.glob(output_locally_reg_neurons_using_dend_axis+exp_name+'*Dendrite.am')[0])
#         angle = Vectors().get_angle_between_vectors(sg_local_avg.graph_data.edge_list[0],\
#                             sg_dend_axis.graph_data.edge_list[0],ignore_opposite_direction=False)
#         if  angle > 45:
#             print(angle)
#             sg = sg_local_avg
#         else:
#             sg = sg_dend_axis
        sg = AmiraSpatialGraph(glob.glob(output_locally_reg_neurons+exp_name+'*Dendrite.am')[0])
        #sg = sg_local_avg
        sg.apply_rotation([[1,0,0],[0,1,0],[0,0,1]],translation=[-i*600,0,0])
        # add pia intersection pt to sg
        top_sg = AmiraSpatialGraph()
        top_sg.graph_data.add_edge([i*600,0,0],[i*600,0,0])

        cell_type_sg.graph_data = cell_type_sg.combine_subgraphs([cell_type_sg.graph_data,sg.graph_data,top_sg.graph_data])
    
    cell_type_sg.write_spatial_graph(output_gallery+'{}_Neurons.am'.format(cell_type))
    

RA20130626*2
RA20130628*2_Cell_1
RA20150807*2_Cell_B
RA20160302*Cell_A
RA20160323*Cell_A
RA20180108*1
RA20180108*2
RA20180109*1_Cell_A
RA20180109*1_Cell_C
RA20180109*2_Cluster_A_Cell_A
RA20180109*2_Cluster_A_Cell_C
RA20180109*2_Cluster_B_Cell_A
RA20180412*2_Cell_B
RA20180412*2_Cell_C
RA20180412*2_Cell_D
RA20180412*2_Cell_E
RA20160307*
RA20130508*1
RA20130513*
RA20130606*
RA20130627*
RA20130627*Cell_2
RA20130627*Cell_3
RA20130319*2
RA20130404*1
RA20130507*2
RA20130605*1
RA20130607*2
RA20130626*1
RA20130628*2_Cell_2
RA20130628*2_Cell_3
RA20130701*2
RA20150810*Cell_B
RA20160302*Cell_B
RA20160308*Cell_A
RA20160310*Cell_A
RA20160322*
RA20160323*Cell_C
RA20180109*1_Cell_B
RA20180109*2_Cluster_B_Cell_B
RA20180109*2_Cluster_B_Cell_C
RA20180109*2_Cluster_B_Cell_D
RA20130722*2
RA20150807*1
RA20150807*2_Cell_A
RA20160324*Cell_A
RA20171214*1_Cell_A
RA20171214*1_Cell_B
RA20180412*2_Cell_A
RA20180411*
RA20150813*2_4x_Contours_forMerge_CELL_B
RA20150813*1
RA20130320*
RA20130603*
RA20130628*
RA2013072

# Get Stats

In [190]:
def get_neuron_stats(exp_name):
    df = pd.DataFrame(columns=df_main.columns)
    
    print(i,exp_name)
    sg = AmiraSpatialGraph(output_surfaces+exp_name+'_barrels.am')
    pia_closed = Surface(output_surfaces+exp_name+'_pia.vtk')
    wm_closed = Surface(output_surfaces+exp_name+'_WM.vtk')
    pia = Surface(output_surfaces+exp_name+'_pia_bottom_open_100.vtk')
    wm = Surface(output_surfaces+exp_name+'_WM_bottom_open_100.vtk')

    #axis_field = wm.create_axis_field(pia,op_sg_name=output_surfaces+exp_name+'_axis_field.am',flip_normals=False,\
    #                                  return_axes=True)
    axis_field = AmiraSpatialGraph(output_axis_field+exp_name+'_axis_field.am',generic_graph=True).graph_data.edge_list
    df['Exp_Name'] = [exp_name]
    df['Pia_Area'] = [pia.get_surface_properties(prop='area')]
    df['WM_Area'] = [wm.get_surface_properties(prop='area')]
    df['Volume'] = [pia_closed.get_surface_properties(prop='volume')]
    vec_lens = []
    for edge in axis_field:
        vec_lens.append(Vectors().get_vec_length(edge))
    df['Cortical_Thickness_Mean'] = [np.array(vec_lens).mean(axis=0)]
    df['Cortical_Thickness_Std'] = [np.array(vec_lens).std(axis=0)]
    df['Cortical_Thickness_CoV'] = [100*np.array(vec_lens).std(axis=0)/np.array(vec_lens).mean(axis=0)]

    is_sticking_out = pia.is_neuron_sticking_out(sg)
    df['Neuron_Sticking_OutOf_Pia'] = [is_sticking_out]
    df['Pia_Intersection_Found'] = [True]
    
    # find neuron axis, nearest axis and angle between them
    #soma_center = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)
    if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
        df['Apical_Dendrite_Present'] = True
    else:
        df['Apical_Dendrite_Present'] = False
    sg = AmiraSpatialGraph(output_neuron_axis+exp_name+'_neuron_axis.am',generic_graph=True)
    neuron_axis = sg.graph_data.edge_list[0]
    
    #angle = Vectors().get_angle_between_vectors(local_avg_axis,dend_center_axis,ignore_opposite_direction=False)
    
    neuron_axis_len = Vectors().get_vec_length(neuron_axis)
    #dend_center_axis_len = Vectors().get_vec_length(dend_center_axis)
    df['Soma_Depth_From_Pia'] = [neuron_axis_len]
    #df['Angle_Between_Axis'] = [angle]
    
    wm_intersection_pt,soma_height = wm.get_vector_intersection_pt(neuron_axis[0],neuron_axis[1],extrapolation_len=-5000)
    if len(wm_intersection_pt) > 0:
        soma_height = Vectors().get_vec_length([wm_intersection_pt,neuron_axis[0]])
        df['Soma_Height_from_WM'] = [soma_height]
        df['Neuron_Axis_Length'] = [neuron_axis_len+soma_height]
        df['WM_Present_Below_Neuron'] = [True]
    else:
        df['WM_Present_Below_Neuron'] = [False]
    
    return df
    

In [191]:
#@delayed
#def parallel_fun(i):
cols = ['Exp_Name','Pia_Area','WM_Area','Volume','Cortical_Thickness_Mean','Cortical_Thickness_Std','Cortical_Thickness_CoV',\
        'Neuron_Sticking_OutOf_Pia','Apical_Dendrite_Present','Pia_Intersection_Found','Soma_Depth_From_Pia','Soma_Height_from_WM'\
        'Neuron_Axis_Length','WM_Present_Below_Neuron']
df_main = pd.DataFrame(columns=cols)

#All_Neurons = AmiraSpatialGraph()
#All_Axes = AmiraSpatialGraph()

for i in range(len(sorted(glob.glob(output_surfaces+'*_pia_bottom_open_100.vtk')))):
    pia_file = sorted(glob.glob(output_surfaces+'*_pia_bottom_open_100.vtk'))[i]
    exp_name = os.path.basename(pia_file)[0:-24]
    #print(exp_name)
    #if exp_name == 'RA20180109_2_Cluster_A_Cell_C_done.am_Segmented_corrected':    
    df = get_neuron_stats(exp_name)
    df_main = df_main.append(df)[df.columns.tolist()]
    #All_Neurons.graph_data = All_Neurons.combine_subgraphs([All_Neurons.graph_data,sg_neuron.graph_data])
    
df_main.to_csv(output_neuron_depth_stats+'Input_Neuron_Stats.csv')
#All_Neurons.write_spatial_graph(output_root+'All_Neurons_Rotated.am')
#All_Axes.write_spatial_graph(output_root+'All_Axes_Rotated.am')


0 RA20130319_2
1 RA20130320_corrected
2 RA20130402
3 RA20130404_1
4 RA20130404_2_corrected
5 RA20130507_1_corrected
6 RA20130507_2_corrected
7 RA20130508_1_corrected
8 RA20130513
9 RA20130603
10 RA20130605_1
11 RA20130606
12 RA20130607_1
13 RA20130607_2
14 RA20130626_1
15 RA20130626_2
16 RA20130627_Cell_2
17 RA20130627_Cell_3
18 RA20130627
19 RA20130628_2_Cell_1
20 RA20130628_2_Cell_2
21 RA20130628_2_Cell_3
22 RA20130628
23 RA20130701_2
24 RA20130722_2
25 RA20130725
26 RA20150804_2_4x-_for_cell_merge_.am_Segmented_corrected
27 RA20150805_1_Cell_A
28 RA20150805_1_Cell_B
29 RA20150806
30 RA20150807_1
31 RA20150807_2_Cell_A
32 RA20150807_2_Cell_B
33 RA20150810_Cell_A
34 RA20150810_Cell_B
35 RA20150811_4x_.am_corrected_Segmented
36 RA20150813_1_Final_Contour_Cell_Merged.am_corrected_Segmented
37 RA20150813_2_4x_Contours_forMerge_CELL_A_.am_corrected_Segmented
38 RA20150813_2_4x_Contours_forMerge_CELL_B_.am_corrected_Segmented
39 RA20160302_Cell_A
40 RA20160302_Cell_B
41 RA20160307_4x_for_m

In [ ]:
# def get_width_height_volume(pts):
#     surf = Surface(pts = pts)
#     bounds = surf.surface.GetBounds()
#     width = bounds[1] - bounds[0]
#     bredth = bounds[3] - bounds[2]
#     height = bounds[5] - bounds[4]
#     volume = width * bredth * height
    
#     return width,bredth,height,volume
    

In [ ]:
# Most neuron specific features are extracted using old code
# Here i only update position specific features. i.e. volume,width and height

# read existing feature csv
# df = pd.read_csv(output_features+'Features.csv')
# for i in df.index:
#     filename = sorted(glob.glob(output_gallery+'*_Neuron.am'))[i]
#     print(filename)
#     sg = AmiraSpatialGraph(filename)
#     total_dend_pts = []
#     # Apical dend stats
#     if len(sg.neuron.dendrite.apical_dendrite.edge_pt_coords) > 0:
#         w,b,h,v = get_width_height_volume(sg.neuron.dendrite.apical_dendrite.edge_pt_coords)
#         df.at[i,'Apical_Width'] = w
#         df.at[i,'Apical_Bredth'] = b
#         df.at[i,'Apical_Height'] = h
#         df.at[i,'Apical_Volume'] = v
#         for pt in sg.neuron.dendrite.apical_dendrite.edge_pt_coords:
#             total_dend_pts.append(pt)
    
#     # basal dend stats
#     w,b,h,v = get_width_height_volume(sg.neuron.dendrite.basal_dendrite.edge_pt_coords)
#     df.at[i,'Basal_Width'] = w
#     df.at[i,'Basal_Bredth'] = b
#     df.at[i,'Basal_Height'] = h
#     df.at[i,'Basal_Volume'] = v
#     for pt in sg.neuron.dendrite.basal_dendrite.edge_pt_coords:
#         total_dend_pts.append(pt)
        
#     # Total Dend stats
#     w,b,h,v = get_width_height_volume(total_dend_pts)
#     df.at[i,'Dendritic_Width'] = w
#     df.at[i,'Dendritic_Bredth'] = b
#     df.at[i,'Dendritic_Height'] = h
#     df.at[i,'Dendritic_Volume'] = v
    
#     if len(sg.neuron.axon.edge_pt_coords) > 0:
#         w,b,h,v = get_width_height_volume(sg.neuron.axon.edge_pt_coords)
#         df.at[i,'Axon_Volume'] = v
    
#     #df['Soma_Depth_From_Pia'] = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)[2]
    

In [ ]:
# df_soma_depth = pd.DataFrame(columns=['Soma_Depth_From_Pia'])
# for i in df.index:
#     filename = sorted(glob.glob(output_gallery+'*_Neuron.am'))[i]
#     print(filename)
#     sg = AmiraSpatialGraph(filename)
#     df_soma_depth.at[i,'Soma_Depth_From_Pia'] = np.array(sg.neuron.soma.edge_pt_coords).mean(axis=0)[2]
    
    

In [ ]:
# df_final = pd.DataFrame()
# df_final = df_final.append(df,ignore_index=True)
# df_final['Soma_Depth_From_Pia'] = df_soma_depth['Soma_Depth_From_Pia']
# df_final.to_csv(output_features+'Features_With_Soma_Depth_V2.csv

In [ ]:
# 

In [ ]:
#')